In [10]:
using Flux, LinearAlgebra, ProgressMeter
include("Architypes.jl")
include("FastNTKMethods.jl")
include("Inspector.jl")

save_to_jld2 (generic function with 1 method)

In [6]:
Nx=500
a=-1.0; b=1.0

xVec=collect(range(a,stop=b,length=Nx));
yVec=sin.(2*pi*xVec) + cos.(xVec) + 0.2*randn(size(xVec)); 


<h1>All models</h1>

In [7]:
SETUP = [
    10          7               5           12;             # depths
    100_000     500_000         20_000      1_000_000;      # number of parameters
    182         514             131         522;            # funnel
    194         557             148         550;            # reverse_funnel
    185         470             99          538;            # hourglass
    168         531             140         489;            # diamond
]

act_func = [σ, relu, tanh, leakyrelu]
act_id = ["sigma", "relu", "tanh", "leakyrelu"]

type = ["block", "funnel", "reverse_funnel", "hourglass", "diamond"]

5-element Vector{String}:
 "block"
 "funnel"
 "reverse_funnel"
 "hourglass"
 "diamond"

In [20]:
N_per_model = 1
dimIN = 1
dimOUT = 1

# block(dimIN, dimOUT, depth, approx_num_params, activations)
# model_architype(architype, dimIN, dimOUT, depth, activation, critical_width)
# kernel(model, x, show_progress = false, diff_mode = 1)

@showprogress for m = 1:length(type)
    for act = 1:length(act_func)
        for d = 1:4
            for n = 1:N_per_model
                if type[m] == "block"
                    M = block(dimIN, dimOUT, SETUP[1,d], SETUP[2,d], act_func[act])
                else
                    M = model_architype(type[m], dimIN, dimOUT, SETUP[1,d], act_func[act], SETUP[m+1,d])
                end
                K = kernel(M, hcat(xVec...), false, 2)
            end
        end
    end
end
